In [1]:
import pandas as pd
import glob
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import numpy as np
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import re
import plotly
import plotly.io as pio
import plotly.express as px
pio.renderers.default = 'notebook'
plotly.offline.init_notebook_mode(connected=True)

In [3]:
# Step 1: Load and concatenate all files with year column added
files = glob.glob('data/madrid_*.csv')
df_list = []
for file in files:
    year_match = re.search(r'madrid_(\d{4})\.csv', file)
    if year_match:
        year = int(year_match.group(1))
        df = pd.read_csv(file, parse_dates=['date'])
        df['year'] = year  # Add year from filename
        df_list.append(df)

# Step 2: Combine them into one DataFrame
data = pd.concat(df_list, ignore_index=True)
data

,date,BEN,CO,EBE,MXY,NMHC,NO_2,NOx,OXY,O_3,PM10,PM25,PXY,SO_2,TCH,TOL,station,year,CH4,NO
0,2009-10-01 01:00:00,NaN,0.27,NaN,NaN,NaN,39.889999,48.150002,NaN,50.680000,18.260000,NaN,NaN,5.55,NaN,NaN,28079003,2009,NaN,NaN
1,2009-10-01 01:00:00,NaN,0.22,NaN,NaN,NaN,21.230000,24.260000,NaN,55.880001,10.580000,NaN,NaN,8.84,NaN,NaN,28079004,2009,NaN,NaN
2,2009-10-01 01:00:00,NaN,0.18,NaN,NaN,NaN,31.230000,34.880001,NaN,49.060001,25.190001,NaN,NaN,6.98,NaN,NaN,28079039,2009,NaN,NaN
3,2009-10-01 01:00:00,0.95,0.33,1.43,2.68,0.25,55.180000,81.360001,1.57,36.669998,26.530001,6.820000,1.3,8.88,1.38,4.62,28079006,2009,NaN,NaN
4,2009-10-01 01:00:00,NaN,0.41,NaN,NaN,0.12,61.349998,76.260002,NaN,38.090000,23.760000,NaN,NaN,7.82,1.41,NaN,28079007,2009,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3808219,2010-08-01 00:00:00,NaN,0.55,NaN,NaN,NaN,125.000000,219.899994,NaN,25.379999,NaN,NaN,NaN,NaN,NaN,NaN,28079056,2010,NaN,NaN
3808220,2010-08-01 00:00:00,NaN,0.27,NaN,NaN,NaN,45.709999,47.410000,NaN,NaN,51.259998,NaN,NaN,7.26,NaN,NaN,28079057,2010,NaN,NaN
3808221,2010-08-01 00:00:00,NaN,NaN,NaN,NaN,0.24,46.560001,49.040001,NaN,46.250000,NaN,NaN,NaN,NaN,1.47,NaN,28079058,2010,NaN,NaN
3808222,2010-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,46.770000,50.119999,NaN,77.709999,NaN,NaN,NaN,NaN,NaN,NaN,28079059,2010,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808224 entries, 0 to 3808223
Data columns (total 20 columns):
 #   Column   Dtype         
---  ------   -----         
 0   date     datetime64[ns]
 1   BEN      float64       
 2   CO       float64       
 3   EBE      float64       
 4   MXY      float64       
 5   NMHC     float64       
 6   NO_2     float64       
 7   NOx      float64       
 8   OXY      float64       
 9   O_3      float64       
 10  PM10     float64       
 11  PM25     float64       
 12  PXY      float64       
 13  SO_2     float64       
 14  TCH      float64       
 15  TOL      float64       
 16  station  int64         
 17  year     int64         
 18  CH4      float64       
 19  NO       float64       
dtypes: datetime64[ns](1), float64(17), int64(2)
memory usage: 581.1 MB


In [5]:
data.describe()

,date,BEN,CO,EBE,MXY,NMHC,NO_2,NOx,OXY,O_3,PM10,PM25,PXY,SO_2,TCH,TOL,station,year,CH4,NO
count,3808224,1.041684e+06,2.651012e+06,1.001724e+06,315415.000000,1.085312e+06,3.787050e+06,2.376275e+06,315695.000000,2.991732e+06,2.861255e+06,816424.000000,315584.000000,2.775960e+06,1.086441e+06,1.038929e+06,3.808224e+06,3.808224e+06,14850.000000,1.532397e+06
mean,2009-06-21 04:25:17.401496064,1.257431e+00,5.503838e-01,1.407927e+00,4.650394,1.873865e-01,5.047151e+01,1.093188e+02,2.280912,3.982616e+01,2.893654e+01,13.738292,2.056178,1.065539e+01,1.435882e+00,5.876842e+00,2.807903e+07,2.008976e+03,1.300849,2.343886e+01
min,2001-01-01 01:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,-31.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.807900e+07,2.001000e+03,0.020000,0.000000e+00
25%,2005-02-10 19:00:00,2.100000e-01,2.600000e-01,3.500000e-01,1.200000,9.000000e-02,2.400000e+01,4.000000e+01,0.960000,1.271000e+01,1.150000e+01,6.450000,0.800000,5.840000e+00,1.300000e+00,1.100000e+00,2.807901e+07,2.005000e+03,1.170000,2.000000e+00
50%,2009-04-11 17:00:00,6.000000e-01,4.000000e-01,8.800000e-01,2.800000,1.500000e-01,4.400000e+01,7.615000e+01,1.320000,3.486000e+01,2.149000e+01,11.000000,1.280000,8.150000e+00,1.380000e+00,3.160000e+00,2.807902e+07,2.009000e+03,1.250000,6.000000e+00
75%,2013-10-17 23:15:00,1.500000e+00,6.500000e-01,1.580000e+00,5.890000,2.400000e-01,6.958000e+01,1.397000e+02,2.740000,5.999000e+01,3.775000e+01,17.670000,2.510000,1.230000e+01,1.510000e+00,6.990000e+00,2.807904e+07,2.013000e+03,1.400000,2.000000e+01
max,2018-05-01 00:00:00,6.639000e+01,1.804000e+01,1.622000e+02,177.600006,9.070000e+00,6.286000e+02,2.537000e+03,103.000000,2.360000e+02,6.950000e+02,506.899994,106.000000,1.991000e+02,1.048000e+01,2.429000e+02,2.807910e+07,2.018000e+03,3.920000,1.146000e+03
std,NaN,1.910831e+00,5.354483e-01,2.146109e+00,5.599223,1.539529e-01,3.455288e+01,1.102871e+02,2.639609,3.039249e+01,2.594859e+01,11.214146,2.395130,9.121267e+00,2.332228e-01,8.524460e+00,2.028574e+01,4.992798e+00,0.195550,5.021504e+01


In [6]:
# Step 3: Merge with station info
stations = pd.read_csv('data/stations.csv')
stations.head()

,id,name,address,lon,lat,elevation
0,28079004,Pza. de España,Plaza de España,-3.712247,40.423853,635
1,28079008,Escuelas Aguirre,Entre C/ Alcalá y C/ O’ Donell,-3.682319,40.421564,670
2,28079011,Avda. Ramón y Cajal,Avda. Ramón y Cajal esq. C/ Príncipe de Vergara,-3.677356,40.451475,708
3,28079016,Arturo Soria,C/ Arturo Soria esq. C/ Vizconde de los Asilos,-3.639233,40.440047,693
4,28079017,Villaverde,C/. Juan Peñalver,-3.713322,40.347139,604


In [7]:
data.head()

,date,BEN,CO,EBE,MXY,NMHC,NO_2,NOx,OXY,O_3,PM10,PM25,PXY,SO_2,TCH,TOL,station,year,CH4,NO
0,2009-10-01 01:00:00,NaN,0.27,NaN,NaN,NaN,39.889999,48.150002,NaN,50.680000,18.260000,NaN,NaN,5.55,NaN,NaN,28079003,2009,NaN,NaN
1,2009-10-01 01:00:00,NaN,0.22,NaN,NaN,NaN,21.230000,24.260000,NaN,55.880001,10.580000,NaN,NaN,8.84,NaN,NaN,28079004,2009,NaN,NaN
2,2009-10-01 01:00:00,NaN,0.18,NaN,NaN,NaN,31.230000,34.880001,NaN,49.060001,25.190001,NaN,NaN,6.98,NaN,NaN,28079039,2009,NaN,NaN
3,2009-10-01 01:00:00,0.95,0.33,1.43,2.68,0.25,55.180000,81.360001,1.57,36.669998,26.530001,6.82,1.3,8.88,1.38,4.62,28079006,2009,NaN,NaN
4,2009-10-01 01:00:00,NaN,0.41,NaN,NaN,0.12,61.349998,76.260002,NaN,38.090000,23.760000,NaN,NaN,7.82,1.41,NaN,28079007,2009,NaN,NaN


In [8]:
# Step 3: Normalize pollutant columns
pollutants = ['BEN', 'CO', 'EBE', 'NMHC', 'NOx', 'NO_2', 'O_3', 'PM10', 'SO_2', 'TCH', 'TOL', 'MXY', 'OXY', 'PXY', 'PM25', 'CH4', 'NO']
scaler = StandardScaler()
scaled_data = data.copy()
scaled_data[pollutants] = scaler.fit_transform(scaled_data[pollutants])

In [17]:
# --- MONTHLY AVERAGE PLOT (across all years) ---
scaled_data['month'] = scaled_data['date'].dt.month

monthly_avg = scaled_data.groupby(['month', 'year'])[pollutants].mean().reset_index()
melted_monthly = monthly_avg.melt(id_vars=['month', 'year'], var_name='Pollutant', value_name='Average')

fig_monthly = px.line(melted_monthly, x='month', y='Average', color='Pollutant', 
                      animation_frame='year', 
                      title='Monthly Average of Pollutants Over Years (Animated)',
                      labels={"month": "Month", "Average": "Average Value"})
fig_monthly.update_layout(template="plotly_white")
fig_monthly.show()

In [10]:
# Step 3: Calculate yearly averages across all stations
yearly_avg_not_scaled = data.groupby('year')[pollutants].mean().reset_index()

# Melt the data for easier plotting with seaborn and plotly
melted_not_scaled = yearly_avg_not_scaled.melt(id_vars='year', var_name='Pollutant', value_name='Average')
# Calculate % change relative to the first year per pollutant
melted_not_scaled['Percentage Change'] = melted_not_scaled.groupby('Pollutant')['Average'].transform(
    lambda x: (x - x.iloc[0]) / x.iloc[0] * 100
)

# --- Dash App ---
app = Dash(__name__)
app.title = "Madrid Pollutants - Percent Change"

app.layout = html.Div([
    html.H2("Madrid Pollutants - % Change Over Time"),
    html.Label("Select Pollutant:"),
    dcc.Dropdown(
        id='pollutant-dropdown',
        options=[{'label': p, 'value': p} for p in melted_not_scaled['Pollutant'].unique()],
        value=melted_not_scaled['Pollutant'].unique()[0]
    ),
    dcc.Graph(id='pollutant-graph')
])

@app.callback(
    Output('pollutant-graph', 'figure'),
    Input('pollutant-dropdown', 'value')
)
def update_graph(selected_pollutant):
    filtered = melted_not_scaled[melted_not_scaled['Pollutant'] == selected_pollutant]
    fig = px.line(
        filtered,
        x='year',
        y='Percentage Change',
        title=f'% Change of {selected_pollutant} Compared to {filtered["year"].min()}',
        markers=True
    )
    fig.update_layout(yaxis_title='% Change', template='plotly_white')
    return fig

if __name__ == '__main__':
    app.run(debug=True)

### 2018 

In [11]:
# Filter for 2018 only
data_2018 = scaled_data[scaled_data['year'] == 2018]

# Group and calculate average by pollutant
avg_2018 = data_2018[pollutants].mean().reset_index()
avg_2018.columns = ['Pollutant', 'Average']

# Calculate percentage contribution
avg_2018['Percentage'] = 100 * avg_2018['Average'] / avg_2018['Average'].sum()


station_pollution = data_2018.groupby('station')[pollutants].mean()
station_pollution['Pollution Index 2018'] = station_pollution.mean(axis=1)
station_pollution = station_pollution.reset_index()
stations.rename(columns={'id': 'station'}, inplace=True)


In [12]:
# First, we will calculate the average pollutant values per station for 2018
station_pollution_2018 = data_2018.groupby('station')[pollutants].mean().reset_index()

station_pollution_2018 = pd.merge(station_pollution_2018, stations, on = 'station', how = 'left')

In [13]:

# Plot the Stacked Bar chart
fig_bar = px.bar(
    station_pollution_2018,
    x='name',
    y=pollutants,
    title="Pollutant Contribution per Station in 2018 (Stacked)",
    labels={"station": "Station", "value": "Pollutant Contribution"},
    color_discrete_sequence=px.colors.qualitative.Set1,  # Use a color scale for better distinction
)

# Update the layout to stack the bars
fig_bar.update_layout(
    barmode='stack',  # Stack the bars
    xaxis_title="Station",
    yaxis_title="Pollutant Contribution",
    template="plotly_white",
)

fig_bar.show()


Question # 3

Area got improved or worst

In [14]:
# Step 1: Filter data for just 2008 and 2018
subset = scaled_data[scaled_data['year'].isin([2008, 2018])]

# Step 2: Compute average Z-score (normalized) for each pollutant per station per year
station_pollution = subset.groupby(['station', 'year'])[pollutants].mean().reset_index()

# Step 3: Compute composite index (mean of all pollutants)
station_pollution['composite'] = station_pollution[pollutants].mean(axis=1)

# Step 4: Pivot to get composite index for 2008 and 2018 side-by-side
pivot = station_pollution.pivot(index='station', columns='year', values='composite').reset_index()
pivot.columns = ['station', '2008', '2018']

# Step 5: Compute change
pivot['composite_change'] = pivot['2018'] - pivot['2008']
pivot = pivot.sort_values('composite_change')

# Step 6: Merge with station coordinates
merged = pivot.merge(stations, on='station', how='left')

# Step 7: Clean missing coordinates or changes
merged_clean = merged.dropna(subset=['composite_change', 'lat', 'lon'])


# Normalize marker size manually to range [5, 30]
min_size, max_size = 5, 30
norm_size = merged_clean['composite_change'].abs()
norm_size = (norm_size - norm_size.min()) / (norm_size.max() - norm_size.min())
merged_clean['marker_size'] = norm_size * (max_size - min_size) + min_size

# # Optional: marker size for visual scaling
# merged_clean['marker_size'] = merged_clean['composite_change'].abs() * 10
# merged_clean['marker_size'] = merged_clean['marker_size'].clip(lower=5, upper=30)

fig_map = px.scatter_mapbox(
    merged_clean,
    lat='lat',
    lon='lon',
    size='marker_size',
    color='composite_change',
    color_continuous_scale='RdYlGn_r',
    hover_name='station',
    title='🗺️ Composite Pollution Change (2008 → 2018)',
    zoom=10,
    height=550
)

fig_map.update_layout(mapbox_style='carto-positron')
fig_map.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig_map.show()

/var/folders/6d/gtg4zywx03dgp5dnsx4rpx4r0000gn/T/ipykernel_17217/643382650.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Location: Position of each marker = Station's geographic location.

Green = Pollution improved (negative change).

Red = Pollution worsened (positive change).

Size: Magnitude of change (absolute change), larger means bigger change, regardless of direction.

Hover info: Useful for seeing station name + specific values.

In [15]:
fig_lollipop = go.Figure()

fig_lollipop.add_trace(go.Scatter(
    x=merged_clean['composite_change'],
    y=merged_clean['name'],
    mode='markers',
    marker=dict(size=10, color=merged_clean['composite_change'], colorscale='RdYlGn_r'),
    name='Change'
))

# Add lines (lollipop stems)
for i, row in merged_clean.iterrows():
    fig_lollipop.add_trace(go.Scatter(
        x=[0, row['composite_change']],
        y=[row['name'], row['name']],
        mode='lines',
        line=dict(color='gray', width=2),
        showlegend=False
    ))

fig_lollipop.update_layout(
    title='🍭 Lollipop Chart: Composite Pollution Change per Station (2008 → 2018)',
    xaxis_title='Composite Index Change (Z-score)',
    yaxis_title='Station',
    template='plotly_white',
    height=800
)

fig_lollipop.show()

Right side of 0 (positive values) → Pollution has worsened at that station.

Left side of 0 (negative values) → Pollution has improved at that station.

Longer line = larger change (good or bad).

## Question 4 

In [16]:
# Filter for 2008–2018
data_decade = scaled_data[scaled_data['year'].between(2008, 2018)]

# Compute yearly averages per pollutant
yearly_avg = data_decade.groupby('year')[pollutants].mean().reset_index()

# Melt for Plotly
melted = yearly_avg.melt(id_vars='year', var_name='Pollutant', value_name='Average')


fig = px.line(
    melted,
    x='year',
    y='Average',
    color='Pollutant',
    title='📈 Normalized Trend of Pollutants (2008–2018)',
    markers=True,
    labels={'Average': 'Normalized Value'}
)

fig.update_layout(template='plotly_white')
fig.show()
